# Código de tratamento dos dados do CAGED

Foram recolhidos dados do número de empregos formais da plataforma CAGED do governo. Os dados estão segmentados por ano, setor, estado e tamanho da empresa.

A classificação do tamanho da empresa foi realizada através do padrão adotado no CAGED, que segue o seguinte formato:

Micro: com até 19 empregados.
Pequena: de 20 a 99 empregados.
Média: 100 a 499 empregados.
Grande: mais de 500 empregados.

Os dados estão disponíveis para consulta no seguinte endereço: bi.mte.gov.br/bgcaged

In [1]:
## Load dependencies.
import os
import pandas as pd
import numpy as np
from global_functions import strip_upper

In [18]:
## Read file.
df_original = pd.read_excel('bases-auxiliares/CAGED.xlsx')

In [ ]:
df = df_original

# 1. Data Wrangling

Nesta etapa iremos limpar a base de dados e transformar os dados para um formato compatível com a base de dados de operações de financiamento.

## 1.1 - Exploração e limpeza do dataset

In [3]:
## Dataset overview.
df_original.head()

,Ano,UF,Setor Gr IBGE,Micro,Pequena,Média,Grande
0,2019,11 - Rondônia,1 - Indústria,9493,10783,8522,12642
1,2019,12 - Acre,1 - Indústria,2372,2540,2837,0
2,2019,13 - Amazonas,1 - Indústria,8308,16948,40635,41806
3,2019,14 - Roraima,1 - Indústria,1415,1426,1080,543
4,2019,15 - Pará,1 - Indústria,16603,23729,26954,42678


In [5]:
## Dataset dimensions.
print(df.shape)

## NA' check.
print(sum(df.isna().sum()))

## Veryfing duplicated records.
print(df.duplicated().sum())

(1458, 7)
0
0


In [6]:
## Checking years.
print(df['Ano'].unique())
print(len(df['Ano'].unique()
))

[2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009 2008 2007 2006
 2005 2004 2003 2002]
18


In [5]:
## Checking UF.
print(df['UF'].unique())

print(len(df['UF'].unique()))


['11 - Rondônia' '12 - Acre' '13 - Amazonas' '14 - Roraima' '15 - Pará'
 '16 - Amapá' '17 - Tocantins' '21 - Maranhão' '22 - Piauí' '23 - Ceará'
 '24 - Rio Grande do Norte' '25 - Paraíba' '26 - Pernambuco'
 '27 - Alagoas' '28 - Sergipe' '29 - Bahia' '31 - Minas Gerais'
 '32 - Espírito Santo' '33 - Rio de Janeiro' '35 - São Paulo'
 '41 - Paraná' '42 - Santa Catarina' '43 - Rio Grande do Sul'
 '50 - Mato Grosso do Sul' '51 - Mato Grosso' '52 - Goiás'
 '53 - Distrito Federal']
27


In [4]:
## Verifying if sequence of UF's repeats through all data
states = df['UF'].unique()

states_seq = states.tolist() * int((len(df)/len(states)))

sum(df['UF'] == states_seq) == len(df)

True

Devido ao padrão de repetição dos estados neste dataset, podemos substituir os estados por uma lista contendo as UF's na mesma sequência:

In [5]:
states_list = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA',
          'MG', 'ES', 'RJ', 'SP', 'PR', 'SC', 'RS', 'MS', 'MT', 'GO', 'DF']

df['UF'] = states_list * (int((len(df)/len(states))))

df.head()


,Ano,UF,Setor Gr IBGE,Micro,Pequena,Média,Grande
0,2019,RO,1 - Indústria,9493,10783,8522,12642
1,2019,AC,1 - Indústria,2372,2540,2837,0
2,2019,AM,1 - Indústria,8308,16948,40635,41806
3,2019,RR,1 - Indústria,1415,1426,1080,543
4,2019,PA,1 - Indústria,16603,23729,26954,42678


In [6]:
## Veryfing sectors.
df['Setor Gr IBGE'].unique()

array(['1 - Indústria', '2 - Construção Civil', '5 - Agropecuária'],
      dtype=object)

In [7]:
## Adjusting names to a standard lower case, no numbers, no spaces and no special characters.
setor = df['Setor Gr IBGE'].tolist()
regex = '^\\d\\s-\\s'


In [8]:
## Inputing adjusted setor to df
df['Setor Gr IBGE'] = strip_upper(setor, regex)
print(df['Setor Gr IBGE'].unique())
df.head()

['INDUSTRIA' 'CONSTRUCAO CIVIL' 'AGROPECUARIA']


,Ano,UF,Setor Gr IBGE,Micro,Pequena,Média,Grande
0,2019,RO,INDUSTRIA,9493,10783,8522,12642
1,2019,AC,INDUSTRIA,2372,2540,2837,0
2,2019,AM,INDUSTRIA,8308,16948,40635,41806
3,2019,RR,INDUSTRIA,1415,1426,1080,543
4,2019,PA,INDUSTRIA,16603,23729,26954,42678


## 1.2 Transformação

Para mesclar os dados com o dataset de financiamento, precisamos deixar esses dados dispostos de uma maneira diferente.

Observando o dataset, percebese que para cada segmento de tamanho da empresa, os dados estão ajustando na coluna. Usando a função melt(), podemos transformar as colunas de tamanho em colunas de categorias de tamanho e valores resultantes.

Desta forma, apenas precisamos ajustar a disposição do dataset para acomodar este novo formato:

In [9]:
## Slicing dataset: Return categorial information and numeric information in separate datasets.
df_cat = df.iloc[:, 0:3] 
df_data = df.iloc[:, 3:]

In [10]:
## Returning df_cat with 4 times (number of categories) its size.
df_cat = pd.concat([df_cat] * 4, ignore_index = True)
len(df_cat)

5832

In [11]:
## Melting numeric data.
df_data = df_data.melt()

In [12]:
len(df_data)

5832

In [13]:
## Joining dataset
df_cat['porte_do_cliente'] = df_data['variable']
df_cat['num_empregados'] = df_data['value']

In [14]:
## Setting header names
df_cat.columns = ['ano', 'uf', 'setor', 'porte_do_cliente', 'num_empregados']

In [15]:
df_cat.tail()

,ano,uf,setor,porte_do_cliente,num_empregados
5827,2002,RS,AGROPECUARIA,Grande,2016
5828,2002,MS,AGROPECUARIA,Grande,0
5829,2002,MT,AGROPECUARIA,Grande,1583
5830,2002,GO,AGROPECUARIA,Grande,664
5831,2002,DF,AGROPECUARIA,Grande,608


In [19]:
## Catching a random row to verify values
print(df_cat.iloc[3672, :])

ano                             2009
uf                                RO
setor               CONSTRUCAO CIVIL
porte_do_cliente               Média
num_empregados                  5741
Name: 3672, dtype: object


In [21]:
## Returning equivalent row in original dataset
df_original.loc[(df_original['Ano'] == 2009) & (df_original['UF'] == '11 - Rondônia')]

,Ano,UF,Setor Gr IBGE,Micro,Pequena,Média,Grande
270,2009,11 - Rondônia,1 - Indústria,9922,12798,6820,5196
756,2009,11 - Rondônia,2 - Construção Civil,2861,4549,5741,14529
1242,2009,11 - Rondônia,5 - Agropecuária,7675,1774,499,0


In [22]:
## Upper porte_do_cliente categories
porte_list = df_cat['porte_do_cliente'].tolist()
df_cat['porte_do_cliente'] = strip_upper(porte_list, regex)

In [24]:
## Overview ajusted dataset
df_cat.tail()

,ano,uf,setor,porte_do_cliente,num_empregados
5827,2002,RS,AGROPECUARIA,GRANDE,2016
5828,2002,MS,AGROPECUARIA,GRANDE,0
5829,2002,MT,AGROPECUARIA,GRANDE,1583
5830,2002,GO,AGROPECUARIA,GRANDE,664
5831,2002,DF,AGROPECUARIA,GRANDE,608


In [25]:
## Exporting adjusted dataset
df_cat.to_csv('bases-auxiliares/CAGED-ajustado.csv',
                   index=False, header=True)
